In [ ]:

#require "core";;
#require "yaml.unix";;
#require "yaml";;
#require "fpath"
#require "yaml.unix"
#require "lwt"
#require "cohttp"
#require "cohttp-lwt-unix"
open Fpath


let s_file = 
"key1: val1
key2: val2"

In [ ]:
Fpath.v "keys.paper.yaml"

In [4]:
let yval = Yaml_unix.of_file_exn (Fpath.v "keys.paper.yaml") 

val yval : Yaml.value =
  `O
    [("urltrade", `String "https://paper-api.alpaca.markets");
     ("urldata", `String "https://data.alpaca.markets");
     ("urlsocket", `String "wss://data.alpaca.markets/stream");
     ("key", `String "PKLDSENB0U8ST5KZI612");
     ("secret", `String "qEmsGIbj8WSBK3DLVDlwcB6KJ3JsLcndnuCHdLIu")]


In [ ]:
type empty = Empty

exception Yaml_list of string
exception Yaml_unknown of string

type _ gvalue = 
  | Vnull : Yaml.value ->   empty gvalue
  | Vfloat : Yaml.value -> float gvalue 
  | Vstring : Yaml.value  -> string gvalue
  | Vlist : Yaml.value -> string gvalue


let val_to_gval : type a. a gvalue -> a  = function
 | Vnull (`Null ) -> Empty
 | Vfloat (`Float x ) -> x
 | Vstring (`String s ) -> s
 | Vlist (`O _) ->  raise (Yaml_list "can't eval yaml list")
 | _ -> raise (Yaml_unknown "unknown yaml value")
 

In [7]:
#use "/home/opam/work/lib/util.ml"

val get_value :
  'a ->
  [> `O of ('a * 'b) list ] ->
  ('b option, [> `Msg of string ]) Rresult.result = <fun>
val get_keys :
  [> `O of ('a * 'b) list ] -> ('a list, [> `Msg of string ]) Rresult.result =
  <fun>
val to_list :
  [> `O of ('a * 'b) list ] -> ('b list, [> `Msg of string ]) Rresult.result =
  <fun>
val get_string : [> `String of 'a ] option -> 'a = <fun>
val get_string_opt : ('a option, 'b) Rresult.result -> 'a option = <fun>
val to_string :
  [> `String of 'a ] -> ('a, [> `Msg of string ]) Rresult.result = <fun>
val to_bool : [> `Bool of 'a ] -> ('a, [> `Msg of string ]) Rresult.result =
  <fun>
val to_float : [> `Float of 'a ] -> ('a, [> `Msg of string ]) Rresult.result =
  <fun>
val map :
  ('a -> 'b) ->
  [> `A of 'a list ] ->
  ([> `A of 'b list ], [> `Msg of string ]) Rresult.result = <fun>


In [11]:
yval |> get_value "key" |> get_string_opt |> get_string 

- : string = "PKLDSENB0U8ST5KZI612"


In [ ]:
let (alpaca_secret, alpaca_key) = 
  let yaml = Yaml_unix.of_file_exn (Fpath.v "keys.paper.yaml") in
  let key = yaml |> get_value "key" |> get_string_opt |> get_string in 
  let secret = yaml |> get_value "secret" |> get_string_opt |> get_string in
  (secret, key)